In [1]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import os 
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
import time

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#functions use inside the model

##### CREATE A CLASS IF EVERYTHING IS FINISHED ####
#more elegant

def D_matrix(W, multifreq = True):
    if multifreq:
        for freq in range(W.shape[0]):
            D[freq, :, :] = torch.diag(W[freq].sum(dim=-1))
    else :
        D = torch.diag(W.sum(dim=-1))
    return D

def laplacian(W, multifreq = True): #here we calculate the non noramlized Laplacian matrix of the graph
    #multifreq means if we get the five frequencies in one or not
    L = D_matrix(W, multifreq = multifreq) - W
    L = L.type(torch.float32)
    return L

def laplacian_norm_mod(W, multifreq = False): #we don't include multifreq option for this function
    D_square = D_square_matrix(W, multifreq = multifreq)
    L_norm_mod = torch.matmul(D_square, torch.matmul(W + torch.eye(W.shape[0]), D_square))
    return L_norm_mod


def x_shaper(x):
    """
    Arg: x as when it outcome from the dataset
    return: x as we wants tensor with (freq, nb_channel, activity)
    """
    x = x.transpose((2, 0, 1))
    return torch.from_numpy(x)


def cheby_laplacian(L, x, theta, multifreq = True):
    """
    Arg: Laplacian matrix, x the extracted signal, theta chebyshev coefficients, K the order of the polynom
    
    we suppose that x was transform to a tensor and that its shape was made as (freq, nb channel, activity)
    
    Return: the sum thetak*Tk(L)*x
    """
 
    y = torch.zeros_like(x) 
    K = theta.shape[0]
    
    if multifreq :
        L_tilde = torch.zeros_like(L) 
        for freq in range(L.shape[0]):
            #chebyshev polynoms is a basis in the domain [-1, 1]
            with torch.no_grad():
                eigenvalues, eigenvectors = torch.linalg.eig(L[freq, :, :])
                eigenvalues = eigenvalues.real #since L is symmetric real L is diagonalizable in the real space
                max_lambda = torch.max(eigenvalues).item() 
            L_tilde[freq, :, :] = ((2*L[freq, :, :]) / max_lambda) - torch.eye(L.shape[1])
            
            x_ = x[freq, :, :]
            L_ = L_tilde[freq, :, :]
            L_.type(torch.float32)

            x0_hat = x_
            x1_hat = torch.matmul(L_, x_)

            if K == 0:
                y[freq, :, :] = x0_hat*theta[0]
            if K == 1:
                y[freq, :, :] = x1_hat*theta[1] + x0_hat*theta[0]
            else:
                y[freq, :, :] = x0_hat*theta[0] + x1_hat*theta[1] 
                for k in range(2, K):
                    x2_hat = 2*torch.matmul(L_, x1_hat) - x0_hat
                    y[freq, :, :] += x2_hat*theta[k]
                    x1_hat = x2_hat
                    x0_hat = x1_hat
    else :
        with torch.no_grad():
            eigenvalues, eigenvectors = torch.linalg.eig(L)
            eigenvalues = eigenvalues.real #since L is symmetric real L is diagonalizable in the real space
            max_lambda = torch.max(eigenvalues).item() 
            
        #rescaling
        L_tilde = ((2*L) / max_lambda) - torch.eye(L.shape[0])
        
        #intialization of variables
        x_ = x
        L_ = L_tilde
        
        L_ = L_.type(torch.float32)

        x0_hat = x_
        x1_hat = torch.matmul(L_, x_)
        
        #calculus of Chebyshev polynoms
        if K == 0:
            y = x0_hat*theta[0]
        if K == 1:
            y = x1_hat*theta[1] + x0_hat*theta[0]
        else:
            y = x0_hat*theta[0] + x1_hat*theta[1] 
            for k in range(2, K):
                x2_hat = 2*torch.matmul(L_, x1_hat) - x0_hat
                y += x2_hat*theta[k]
                x1_hat = x2_hat
                x0_hat = x1_hat              
    return y



def polynomial_approximation(L, x, theta, multifreq = True):
    """
    Arg: Laplacian matrix, x the extracted signal, theta chebyshev coefficients, K the order of the polynom
    
    we suppose that x was transform to a tensor and that its shape was made as (freq, nb channel, activity)
    
    Return: the U*sum theta[k]*lambda**k*U^{T}
    """
 
    y = torch.zeros_like(x) 
    K = theta.shape[0]
    
    if multifreq :
        L_tilde = torch.zeros_like(L) 
        for freq in range(L.shape[0]):
            with torch.no_grad():
                eigenvalues, eigenvectors = torch.linalg.eig(L[freq, :, :])
                eigenvalues = eigenvalues.real #since L is symmetric real L is diagonalizable in the real space
                U = eigenvectors.real
        
            eigen_values_diag = torch.diag(eigenvalues)
            aux = torch.zeros_like(eigen_values_diag)
            for k in range(K):
                aux += torch.linalg.matrix_power(eigen_values_diag, k)*theta[k]
            y[freq, :, :] = torch.matmul(torch.matmul(U, torch.matmul(aux, U.transpose(1, 0))), x)
            
    else :  
        #since we don't work with chebyshev approximation there is no need to rescale the Laplacian
        with torch.no_grad():
                eigenvalues, eigenvectors = torch.linalg.eig(L)
                eigenvalues = eigenvalues.real #since L is symmetric real L is diagonalizable in the real space
                U = eigenvectors.real
        
        eigen_values_diag = torch.diag(eigenvalues)
        aux = torch.zeros_like(eigen_values_diag)
        for k in range(K):   
            aux += torch.linalg.matrix_power(eigen_values_diag, k)*theta[k]
        y = torch.matmul(torch.matmul(U, torch.matmul(aux, U.transpose(1, 0))), x) 
    return y



def train_model(model, num_epochs, train_loader, test_loader, optimizer, loss_function, alpha):

    #training of the model
    train_loss = []
    valid_loss = []
    train_accuracy = []
    valid_accuracy = []
    

    for epoch in range(num_epochs):

        ############################
        # Train
        ############################

        iter_loss = 0.0
        correct = 0
        iterations = 0

        model.train()        # Put the network into training mode

        for i, (signal, label_) in enumerate(train_loader):

            optimizer.zero_grad()     # Clear off the gradients from any past operation
            outputs = model(signal)      # Do the forward pass
            loss = loss_function(outputs, label_) # Calculate the loss
            iter_loss += loss.item() # Accumulate the loss

            #L1-Regularization
            l1_norm = sum(abs(p).sum() for p in model.parameters())
            loss = loss + alpha*l1_norm
            loss.backward()           # Calculate the gradients with help of back propagation
           #print('W')
           #print(model.GCNLayer1.W)
           #print(model.GCNLayer1.W.grad)
           #print('theta')
           #print(model.GCNLayer1.theta)
           #print(model.GCNLayer1.theta.grad)
            optimizer.step()          # Adjust the parameters based on the gradients


            # Record the correct predictions for training data 
            predicted = torch.max(outputs.data, 1)[1]
            correct += (predicted == label_).sum()
            iterations += 1

        # Record the training loss
        train_loss.append(iter_loss / iterations)
        # Record the training accuracy
        train_accuracy.append((100 * correct / float(len(train_dataset))))


        ############################
        # Test
        ############################

        loss = 0.0
        correct = 0
        iterations = 0

        model.eval()                    # Put the network into evaluate mode

        for i, (signal, label_) in enumerate(test_loader):

            outputs = model(signal)      # Do the forward pass
            loss += criterion(outputs, label_).item() # Calculate the loss

            # Record the correct predictions for training data
            predicted = torch.max(outputs.data, 1)[1]
            correct += (predicted == label_).sum()

            iterations += 1

        # Record the validation loss
        valid_loss.append(loss / iterations)
        # Record the validation accuracy
        correct_scalar = np.array([correct.clone()])[0]
        valid_accuracy.append(correct_scalar.item() / len(test_dataset) * 100.0)
        
    return train_loss, valid_loss, train_accuracy, valid_accuracy

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class DGCNLayerCheb(nn.Module):
    def __init__(self, W, K):
        super().__init__()
        self.W = nn.Parameter(W)
        self.theta = nn.Parameter(torch.FloatTensor(K, 1))
        self.theta = nn.init.uniform_(self.theta, 0, 10)
        
    def forward(self, x):
        L = laplacian(self.W, multifreq = False)
        X = cheby_laplacian(L, x, self.theta, multifreq = False)
        return X
    
class DGCN_Cheb(nn.Module):
    def __init__(self, W, K, input_size, hidden_size, output_size):
        super().__init__()
        self.GCNLayer1 = DGCNLayerCheb(W, K)
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=0.3)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.GCNLayer1(x) 
        x = F.relu(x)
        x = x.view(-1, self.input_size)
        x = self.l1(x)
        x = self.tanh(x)
        x = self.dropout(x)
        x = self.l2(x)
        return self.tanh(x)
    

class DGCNLayerPoly(nn.Module):
    def __init__(self, W, K):
        super().__init__()
        self.W = nn.Parameter(W)
        self.theta = nn.Parameter(torch.FloatTensor(K, 1))
        self.theta = nn.init.uniform_(self.theta, 0, 10)
        
    def forward(self, x):
        L = laplacian(self.W, multifreq = False)
        X = polynomial_approximation(L, x, self.theta, multifreq = False)
        return X
    
class DGCN_Poly(nn.Module):
    def __init__(self, W, K, input_size, hidden_size, output_size):
        super().__init__()
        self.DGCNLayer1 = DGCNLayerPoly(W, K)
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=0.2)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.DGCNLayer1(x) 
        x = F.relu(x)
        x = x.view(-1, self.input_size)
        x = self.l1(x)
        x = self.tanh(x)
        x = self.dropout(x)
        x = self.l2(x)
        return self.tanh(x)
    
    
class DGCNLayerPolyNormMod(nn.Module):
    def __init__(self, W, K):
        super().__init__()
        self.W = nn.Parameter(W)
        self.theta = nn.Parameter(torch.FloatTensor(K, 1))
        self.theta = nn.init.uniform_(self.theta, 0, 10)
        
    def forward(self, x):
        L = laplacian_norm_mod(self.W, multifreq = False)
        X = polynomial_approximation(L, x, self.theta, multifreq = False)
        return X
   
#ssks
class DGCN_PolyNormMod(nn.Module):
    def __init__(self, W, K, input_size, hidden_size, output_size):
        super().__init__()
        self.GCNLayer1 = DGCNLayerPolyNormMod(W, K)
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=0.2)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.GCNLayer1(x) 
        x = F.relu(x)
        x = x.view(-1, self.input_size)
        x = self.l1(x)
        x = self.tanh(x)
        x = self.dropout(x)
        x = self.l2(x)
        return self.tanh(x)
    
class DGCNLayerChebNormMod(nn.Module):
    def __init__(self, W, K):
        super().__init__()
        self.W = nn.Parameter(W)
        self.theta = nn.Parameter(torch.FloatTensor(K, 1))
        self.theta = nn.init.uniform_(self.theta, 0, 10)
        
    def forward(self, x):
        L = laplacian_norm_mod(self.W, multifreq = False)
        X = cheby_laplacian(L, x, self.theta, multifreq = False)
        return X
    
class DGCN_ChebNormMod(nn.Module):
    def __init__(self, W, K, input_size, hidden_size, output_size):
        super().__init__()
        self.GCNLayer1 = DGCNLayerChebNormMod(W, K)
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=0.4)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.GCNLayer1(x) 
        x = F.relu(x)
        x = x.view(-1, self.input_size)
        x = self.l1(x)
        x = self.tanh(x)
        x = self.dropout(x)
        x = self.l2(x)
        return self.tanh(x)

In [5]:
#Useful functions

def import_matfiles(path):
    list_mat_ex = os.listdir(path)
    #list_mat_file contains all interessant files for our research
    list_mat_ex.remove('readme.txt')
    #list_mat_file contains all interessant files for our research
    label = list_mat_ex[-1]
    trash = list_mat_ex.pop(-1)
    return list_mat_ex, label
    
def find_max_shape(datas, freq = None):
    #shape of data in datas (freq, nb channel, activity)
    if freq == None:
        max_ = datas[0].shape[2]
        for data in datas[1:]:
            act_shape = data.shape[2]
            if act_shape > max_ :
                max_ = act_shape
    else :
        #hape of data in datas (nb channel, activity)
        max_ = datas[0].shape[1]
        for data in datas[1:]:
            act_shape = data.shape[1]
            if act_shape > max_ :
                max_ = act_shape
    return max_  

def x_shaper(x):
    """
    Arg: x as when it outcome from the dataset
    return: x as we wants tensor with (freq, nb_channel, activity)
    """
    x = x.transpose((2, 0, 1))
    return torch.from_numpy(x).type(torch.float32)

def datas_shaper(datas):
    res = []
    for data in datas:
        print(data.shape)
        res.append(x_shaper(data))
    return res 

def padd(datas, freq = None):
    #shape of data in datas (freq, nb channel activity)
    max_shape = find_max_shape(datas, freq =freq)
    res = []
    if freq == None :
        for data in datas:
            aux = torch.zeros((data.shape[0], data.shape[1], max_shape))
            if data.shape[2] < max_shape:
                add_shape = max_shape - data.shape[2]
                for freq in range(data.shape[0]):
                    pad_tensor = torch.zeros((data.shape[1], add_shape))
                    aux[freq, :, :] = torch.cat((data[freq, :, :], pad_tensor), dim=1)
                    aux = aux.type(torch.float32)
                res.append(aux)
            else :
                res.append(data.type(torch.float32))
    else :
        for data in datas:
            aux = torch.zeros((data.shape[0], max_shape))
            if data.shape[1] < max_shape:
                add_shape = max_shape - data.shape[1]
                pad_tensor = torch.zeros((data.shape[0], add_shape))
                aux = torch.cat((data, pad_tensor), dim=1)
                aux = aux.type(torch.float32)
                res.append(aux)
            else :
                res.append(data.type(torch.float32))
    return res

def normalize(x, freq = None):
    if freq == None:
        for frequencies in range(x.shape[0]):
            mean_ = torch.mean(x[frequencies, :, :], 1).reshape(-1, 1)
            std_ = torch.std(x[frequencies, : ,: ], 1).reshape(-1, 1)
            x[frequencies, :, :] = (x[frequencies, :, :] - mean_) / std_
            return x
    else :
        mean_ = torch.mean(x[freq, :, :], 1).reshape(-1, 1)
        std_ = torch.std(x[freq, : ,: ], 1).reshape(-1, 1)
        x[freq, :, :] = (x[freq, :, :] - mean_) / std_
        return x[freq, :, :]

In [6]:
#version to improve in a near future..
class SignalDataset(Dataset):
    def __init__(self, path, signals, subject_number, experiment_number, labels, smoothing_method, freq = None):
        """
        Args: 
            path: path to folder with all the .mat files of the dataset
            signals: feature that we want to extract
            subject_number: subject of the experimenr
            epxeriment_number: number of the experiment, like 0 week 1 week or 2 week 
            (MAYBE I MISUNDERSTOOD THIS PART READ AGAIN DATASET DETAILS)
            labels: labels list
            smoothing method: movingAve or LDS
            freq: 0, 1, 2, 3 or 4 for just specific one and None if you want them all
        """
        list_file, _ = import_matfiles(path)
        #selection of interesting files that means the one related to the subject
        list_sub_file = []
        for file in list_file:
            num = file.split('_')[0]
            if int(num) == subject_number: 
                list_sub_file.append(file)
             
        working_file = list_sub_file[experiment_number]
        
        dic = loadmat(path + '\\' + working_file) 
        if smoothing_method == 'movingAve':
            smooth = signals.lower() + '_movingAve'
        elif smooting_method == 'LDS':
            smooth = signals.lower() + '_LDS'
        else :
            raise ValueError('Please select a good smoothing method: movingAve or LDS')
        
        #maybe a list is not the best tool to store it
        datas = []
        for k in range(15):
            sig = dic[smooth + str(k+1)]
            sig = x_shaper(sig)
            sig = normalize(sig, freq = freq) #change the freq if needed!!!!!!!!!!!!
            datas.append(sig)
            
        
        datas = padd(datas, freq = freq) #everything is happening in this function, the changement from Long to Float
                                    #and the slicing for the frequency we want

        labels = labels.reshape(-1, )
        labels = labels + 1 #sliding to use CrossEntropy Pytorch function
        labels = torch.from_numpy(labels)
                                  
                                  
        self.datas = datas
        self.labels = labels
    
    def __len__(self):
        return len(self.datas)
    
    def __getitem__(self, idx):
        return self.datas[idx], self.labels[idx].item()

## Creation of adjency matrices for every features

In [7]:
#channels according to the index in the dataset
channel = ['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 
                'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6','FT8','T7','C5','C3','C1',
                'CZ','C2','C4','C6', 'T8','TP7','CP5','CP3','CP1','CPZ','CP2','CP4','CP6','TP8','P7',
                'P5','P3','P1','PZ','P2','P4','P6','P8','PO7','PO5','PO3','POZ','PO4','PO6','PO8','CB1',
                'O1','OZ','O2','CB2']
idx = np.arange(62)
df_channel = pd.DataFrame({'idx': idx, 'channel': channel })


In [9]:
#initialization of the adjency matrix for DE and PSD features
feature_number = 62
W = np.zeros((5, feature_number, feature_number))
for freq in range(5):
    #I don't see other way than initialiazing W manually
    #was done accodingly to the EEG channel map and order, hope to don't have make mistake
    #FP1
    W[freq, 0, 1] = 1
    W[freq, 0, 3] = 1
    #FPZ
    W[freq, 1, 0] = 1
    W[freq, 1, 2] = 1
    #FP2
    W[freq, 2, 1] = 1
    W[freq, 2, 4] = 1
    #AF3
    W[freq, 3, 8] = 1
    W[freq, 3, 7] = 1
    W[freq, 3, 6] = 1
    W[freq, 3, 0] = 1
    #AF4
    W[freq, 4, 2] = 1
    W[freq, 4, 10] = 1
    W[freq, 4, 11] = 1
    W[freq, 4, 12] = 1
    #F7
    W[freq, 5, 6] = 1
    W[freq, 5, 14] = 1
    #F5
    W[freq, 6, 3] = 1
    W[freq, 6, 5] = 1
    W[freq, 6, 15] = 1
    W[freq, 6, 7] = 1
    #F3
    W[freq, 7, 3] = 1
    W[freq, 7, 8] = 1
    W[freq, 7, 16] = 1
    #F1
    W[freq, 8, 9] = 1
    W[freq, 8, 17] = 1
    #FZ
    W[freq, 9, 10] = 1
    W[freq, 9, 18] = 1
    #F2
    W[freq, 10, 11] = 1
    W[freq, 10, 19] = 1
    #F4
    W[freq, 11, 12] = 1
    W[freq, 11, 20] = 1
    #F6 
    W[freq, 12, 13] = 1
    W[freq, 12, 21] = 1
    #F8
    W[freq, 13, 22] = 1
    #FT7
    W[freq, 14, 15] = 1
    W[freq, 14, 23] = 1
    #FC5 to TP8
    for k in range(15, 41): #till TP8 (number 40) we have two new neigbors: on the right and under +1 and +9 (nine electrodes per line)
        if k == 22 or k == 31 or k == 40 :
            W[freq, k, k+9] = 1
        else:
            W[freq, k, k+1] = 1
            W[freq, k, k+9] = 1
    #P7
    W[freq, 41, 42] = 1
    W[freq, 41, 50] = 1
    #P5
    W[freq, 42, 43] = 1
    W[freq, 42, 51] = 1
    #P3
    W[freq, 43, 44] = 1
    #P1
    W[freq, 44, 45] = 1
    W[freq, 44, 52] = 1
    #PZ
    W[freq, 45, 46] = 1
    W[freq, 45, 53] = 1
    #P2
    W[freq, 46, 47] = 1
    W[freq, 46, 54] = 1
    #P4
    W[freq, 47, 48] = 1
    #P6
    W[freq, 48, 49] = 1
    W[freq, 48, 55] = 1
    #P8
    W[freq, 49, 56] = 1
    #PO7
    W[freq, 50, 51] = 1
    W[freq, 50, 57] = 1
    #PO5
    W[freq, 51, 52] = 1
    W[freq, 50, 57] = 1
    #PO3
    W[freq, 52, 53] = 1
    W[freq, 52, 58] = 1
    #POZ
    W[freq, 53, 54] = 1
    W[freq, 53, 59] = 1
    #PO4
    W[freq, 54, 55] = 1
    W[freq, 54, 60] = 1
    #PO6
    W[freq, 55, 56] = 1
    W[freq, 55, 61] = 1
    #PO8
    W[freq, 56, 61] = 1
    #CB1
    W[freq, 57, 58] = 1
    #O1
    W[freq, 58, 59] = 1
    #OZ
    W[freq, 59, 60] = 1
    #O2
    W[freq, 60, 61] = 1

for freq in range(5):
    for i in range(feature_number):
        for j in range(i, feature_number):
            if W[freq, j, i] == 0:
                W[freq, j, i] = W[freq, i, j]
    W[freq, :, :] = W[freq, :, :] - np.eye(feature_number)

#for some training stability reasons we initialized the diagonal of W with 1
for freq in range(5):
    W[freq, :, :] = W[freq, :, :] + np.eye(W.shape[1], dtype=int)

W_de = torch.from_numpy(W).type(torch.float32)
print(W_de.shape)

torch.Size([5, 62, 62])


In [10]:
#initialization for DASM and RASM 
#WE ARE GONNA TAKE A TRUNCATED FORM OF THE DE ADJENCY MATRIX 
#take the same pattern which is a repeated one 

W_dasm = torch.zeros(5, 27, 27)
for freq in range(5):
    W_dasm[freq, :, :] = W_de[0, :27, :27] #we take 0 because it's all the same for every frequencies
print(W_dasm.shape)

#initialization for DCAU
W_dcau = torch.zeros(5, 23, 23)
for freq in range(5):
    W_dcau[freq, :, :] = W_de[0, :23, :23] #we take 0 because it's all the same for every frequencies
print(W_dcau.shape)

#intialization for ASM 
W_asm = torch.zeros(5, 54, 54)
for freq in range(5):
    W_asm[freq, :, :] = W_de[0, :54, :54] #we take 0 because it's all the same for every frequencies
print(W_asm.shape)

torch.Size([5, 27, 27])
torch.Size([5, 23, 23])
torch.Size([5, 54, 54])


In [11]:
#function to loop over the entire dataset
def get_accuracy_results(feature, W_feature, model_name):
    feature_list = ['de', 'psd', 'dasm', 'rasm', 'dcau', 'asm']
    if feature not in feature_list:
        raise TypeError('Please select a feature in ' + str(feature_list))
    
    path = r'C:\Users\harol\UsefulCode\CP_DSAI\Project\Dataset\SEED\ExtractedFeatures'
    signals = feature
    
    l, label_ = import_matfiles(path)
    labels = loadmat(path + '\\' + label_)['label']
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    smoothing_method = 'movingAve'
    batch_size = 1
    learning_rate = 0.0001
    alpha = 1e-7
    K = 10
    num_epochs = 1000

    glob_test_acc = []

    for freq in range(4): #5 frequencies used across the dataset
        freq_test_acc = []
        for sub in range(1, 16):#15 subject numbered form 1 to 15
            for exp in range(3):   #3 experiment for each subject
                dataset = SignalDataset(path, signals, sub, exp, 
                                        labels, smoothing_method, freq = freq)
                train_dataset, test_dataset = random_split(dataset, [9, 6])
                train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, 
                                                           shuffle = True)
                test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = True)
    
                W_ = W_feature[freq].type(torch.float32).clone().detach().requires_grad_(True) 

                model = choose_model(model_name, device, W_, K, feature).to(device)
                criterion = nn.CrossEntropyLoss()
                optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)

                train_loss, valid_loss, train_accuracy, valid_accuracy = train_model(model, num_epochs, 
                                                                    train_loader, test_loader, optimizer, 
                                                                             criterion, alpha)
                freq_test_acc.append(valid_accuracy)
                
                print('frequencies: {}, sub: {}, exp: {}'.format(freq, sub, exp))
                print('Last Validation set accuracy score '+ str(valid_accuracy[-1]))

        glob_test_acc.append(freq_test_acc)
    
    
    return glob_test_acc

def eh_merce_function(feature, glob_test_acc):
    frequencies = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    res= []
    for freq in range(len(glob_test_acc)):
        
        aux = np.array(glob_test_acc[freq])
        
        
        print('Feature: {}'.format(feature))
        print('Frequency: ' + freq)
        print('')
        
        print('------- Best validation accuracy got -----------')
        best_val_acc_value = np.max(np.array(aux[0]))
        best_val_acc_epoch = np.argmax(np.array(aux[0]))
        best_val_acc_subexp = 0
        for k in range(1, aux.shape[0]):
            best_val_acc_value_aux = np.max(np.array(aux[k]))
            if best_val_acc_value_aux > best_val_acc_value :
                best_val_acc_value = best_val_acc_value_aux
                best_val_acc_epoch = np.argmax(aux[k])
                best_val_acc_subexp = k
                
        print('The best validation accuracy got is {} for the experiment {} of the subject {} at the epoch {}'.format(best_val_acc_value, (best_val_acc_subexp  % 3) + 1, (best_val_acc_subexp  // 3) + 1, best_val_acc_epoch ))
        print('')

        
        print('------ Best validation accuracy got for each experiment ----------')
        pd_idx = []
        for sub in range(1, 16):
            for exp in range(1, 4):
                pd_idx.append('subject: ' + str(sub) + ', experiment: ' + str(exp))
        df_m = pd.DataFrame(np.max(aux, axis = 1), index = pd_idx, columns = [''])
        print(df_m)
        print('')
        
        print('------ Mean of the validation accuracy for the last 50 epochs-----------')
        df_mean = pd.DataFrame(np.mean(aux[:, 950:], axis = 1).reshape(-1, 1), 
                                               index = pd_idx, columns = [''])
        print(df_mean)
        
        
        
        mean_ = np.mean(aux)
        std_ = np.std(aux)
        
        res.append((mean_, std_))
        print('')
        print('----MEAN/STD-----')
        print((mean_, std_))
    return res

#model = DGCN_Cheb(W_, K, 62*265, 50, 3).to(device)
def choose_model(model_name, device, W, K, feature):
    
    feature_list = ['de', 'psd', 'dasm', 'rasm', 'dcau', 'asm']
    if feature not in feature_list:
        raise TypeError('Please select a feature in ' + str(feature_list))
        
    if (feature == 'de') or (feature == 'psd'):
        if model_name == 'Chebyshev':
            model = DGCN_Cheb(W, K, 62*265, 50, 3).to(device)
        elif model_name == 'Poly':
            model = DGCN_Poly(W, K, 62*265, 50, 3).to(device)
        elif model_name == 'ChebyshevNormMod':
            model = DGCN_ChebNormMod(W, K, 62*265, 50, 3).to(device)
        elif model_name == 'ChebyshevGCN':
            model = GCNCheb(W, K, 62*265, 50, 3).to(device)
        else :
            raise ValueError('Please select Cheby, Poly, ChebyshevNormMod, PolyNormMod')
    
    if (feature == 'dasm') or (feature == 'rasm'):
        if model_name == 'Chebyshev':
            model = DGCN_Cheb(W, K, 27*265, 50, 3).to(device)
        elif model_name == 'Poly':
            model = DGCN_Poly(W, K, 27*265, 50, 3).to(device)
        elif model_name == 'ChebyshevNormMod':
            model = DGCN_ChebNormMod(W, K, 27*265, 50, 3).to(device)
        elif model_name == 'ChebyshevGCN':
            model = GCNCheb(W, K, 27*265, 50, 3).to(device)
        else :
            raise ValueError('Please select Cheby, Poly, ChebyshevNormMod, PolyNormMod')
            
    if (feature == 'dcau'):
        if model_name == 'Chebyshev':
            model = DGCN_Cheb(W, K, 23*265, 50, 3).to(device)
        elif model_name == 'Poly':
            model = DGCN_Poly(W, K, 23*265, 50, 3).to(device)
        elif model_name == 'ChebyshevNormMod':
            model = DGCN_ChebNormMod(W, K, 23*265, 50, 3).to(device)
        elif model_name == 'ChebyshevGCN':
            model = GCNCheb(W, K, 23*265, 50, 3).to(device)
        else :
            raise ValueError('Please select Cheby, Poly, ChebyshevNormMod, PolyNormMod')
            
    if (feature == 'asm'):
        if model_name == 'Chebyshev':
            model = DGCN_Cheb(W, K, 54*265, 50, 3).to(device)
        elif model_name == 'Poly':
            model = DGCN_Poly(W, K, 54*265, 50, 3).to(device)
        elif model_name == 'ChebyshevNormMod':
            model = DGCN_ChebNormMod(W, K, 54*265, 50, 3).to(device)
        elif model_name == 'ChebyshevGCN':
            model = GCNCheb(W, K, 54*265, 50, 3).to(device)
        else :
            raise ValueError('Please select Cheby, Poly, ChebyshevNormMod, PolyNormMod')
    
    
    return model

In [12]:
print(type(W_de), type(W_dasm), type(W_dcau), type(W_asm))

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
model_names = ['Chebyshev', 'Poly', 'ChebyshevNormMod', 'ChebyshevGCN']

for model_name in model_names:
    
    print('-----------Model: {}--------------'.format(model_name))
    print('')
    
    #ASM
    t1 = time.time()
    asm_glob_test_acc = get_accuracy_results('asm', W_asm, model_name)
    print('-------Temps d\'exécution -----------')
    print(t2 - t1)
    print('')
    res_asm = eh_merce_function('asm', asm_glob_test_acc)
    
    #DCAU
    t1 = time.time()
    dcau_glob_test_acc = get_accuracy_results('dcau', W_dcau, model_name)
    print('-------Temps d\'exécution -----------')
    print(t2 - t1)
    print('')
    res_dcau = eh_merce_function('dcau', dcau_glob_test_acc)
    print('')
    
    #DE
    t1 = time.time()
    de_glob_test_acc = get_accuracy_results('de', W_de, model_name)
    t2 = time.time()
    print('-------Temps d\'exécution -----------')
    print(t2 - t1)
    print('')
    res_de = eh_merce_function('de', de_glob_test_acc)
    print('')

    #PSD
    t1 = time.time()
    psd_glob_test_acc = get_accuracy_results('psd', W_de, model_name)
    print('-------Temps d\'exécution -----------')
    print(t2 - t1)
    print('')
    res_psd = eh_merce_function('psd', psd_glob_test_acc)
    print('')

    #DASM
    t1 = time.time()
    dasm_glob_test_acc = get_accuracy_results('dasm', W_dasm, model_name)
    print('-------Temps d\'exécution -----------')
    print(t2 - t1)
    print('')
    res_dasm = eh_merce_function('dasm', dasm_glob_test_acc)
    print('')

    #RASM
    t1 = time.time()
    rasm_glob_test_acc = get_accuracy_results('rasm', W_dasm, model_name)
    print('-------Temps d\'exécution -----------')
    print(t2 - t1)
    print('')
    res_rasm = eh_merce_function('rasm', rasm_glob_test_acc)
    print('')


-----------Model: Chebyshev--------------

